In [1]:
import duckdb
import pandas as pd
import numpy as np

In [2]:
# open connection to duck db at election/election.db 
conn = duckdb.connect('election/election.db', read_only=True)

In [3]:
# list all tables in the database
tables = conn.execute("SHOW TABLES").fetchdf()
print("Tables in the database:")
print(tables)

Tables in the database:
                                             name
0                                    constituency
1          current_first_past_the_post_candidates
2        current_proportional_election_candidates
3                          dim_current_candidates
4             dim_current_proportional_candidates
5                        dim_fptp_2079_candidates
6                             dim_fptp_candidates
7                          dim_parliament_members
8                                     dim_parties
9                             dim_past_candidates
10                                      districts
11                 parliament_member_name_mapping
12                             parliament_members
13                 party_current_new_name_mapping
14  past_2074_first_past_the_post_election_result
15         past_2074_proportional_election_result
16  past_2079_first_past_the_post_election_result
17         past_2079_proportional_election_result
18       past_first_past_t

In [4]:
dim_fptp_candidates = conn.execute("SELECT * FROM dim_fptp_candidates").fetchdf()
print("\ndim_fptp_candidates table:")
dim_fptp_candidates.head()


dim_fptp_candidates table:


,candidate_id,candidate_name,gender,age,age_group,date_of_birth,father_name,spouse_name,qualification,qualification_level,...,is_educated,is_uneducated,is_gen_z,is_grandpa,is_influential,is_opportunist,is_split_vote_candidate,is_loyal,candidate_type,tags
0,339403,राजेन्द्र प्रसाद लिङ्देन,पुरुष,60,60+,60,मान प्रसाद लिङ्देन,सिता थापा,स्‍नातक,Bachelor,...,True,False,False,False,False,False,False,True,Same Location,"[Educated, Loyal]"
1,339725,राजेन्द्र कुमार राई,पुरुष,59,50-60,59,तेज वहादुर राई,मन्जु राई,आई.ए.,2,...,True,False,False,False,False,False,False,True,Same Location,"[Educated, Loyal]"
2,340452,ज्ञानेन्द्र बहादुर कार्की,पुरुष,69,60+,69,कृष्ण मोहन कार्की,नविना राणा,स्नातोकत्तर(राजनैतिक शास्त्र),Masters,...,True,False,False,True,False,False,False,True,Same Location,"[Tourist, Educated, New Party, Grandpa, Loyal]"
3,340917,उपेन्द्र यादव,पुरुष,65,60+,65,धनि लाल यादव,पार्वती देवी यादव,स्नातकोत्तर,Masters,...,True,False,False,True,False,False,False,False,Chheparo,"[Tourist, Chheparo, Vaguwa, Educated, Grandpa]"
4,340864,लिलानाथ श्रेष्ठ,पुरुष,68,60+,68,जित वहादुर श्रेष्ठ,विश्व श्रेष्ठ,स्नातक,Bachelor,...,True,False,False,True,False,False,False,True,Same Location,"[Educated, Grandpa, Loyal]"


In [7]:
dim_fptp_candidates.head().to_json('dim_fptp_candidates_sample.json', orient='records', lines=True, force_ascii=False, indent=2)

In [5]:
dim_parties = conn.execute("SELECT * FROM dim_parties").fetchdf()
dim_parties

,party_id,current_party_name,previous_names
0,1,राष्ट्रिय नागरिक पार्टी,[]
1,2,आम जनता पार्टी(एकल चुनाव चिन्ह),[]
2,3,इतिहासिक जनता पार्टी,[इतिहासिक प्रजातान्त्रिक जनता पार्टी नेपाल]
3,4,उज्यालो नेपाल पार्टी,[]
4,5,गतिशील लोकतान्त्रिक पार्टी,[]
...,...,...,...
60,61,संयुक्त नागरिक पार्टी,"[नेपाल दलित पार्टी(एकल चुनाव चिन्ह), सामाजिक ए..."
61,62,समावेशी समाजवादी पार्टी,[]
62,63,समावेशी समाजवादी पार्टी नेपाल,[]
63,64,सार्वभौम नागरिक पार्टी,[]


In [ ]:
qualification_level_mapping = conn.execute("SELECT * FROM qualification_level_mapping").fetchdf()
qualification_level_mapping

In [9]:
# load stg_current_candidates into a pandas dataframe
stg_current_candidates = conn.execute("SELECT * FROM stg_current_candidates").fetchdf()
stg_current_candidates.head()

,CandidateID,CandidateName,AGE_YR,Gender,PoliticalPartyName,SYMBOLCODE,SymbolName,CTZDIST,DistrictName,StateName,...,R,E_STATUS,DOB,FATHER_NAME,SPOUCE_NAME,QUALIFICATION,NAMEOFINST,EXPERIENCE,OTHERDETAILS,ADDRESS
0,339933,क्षितिज थेबे,38,पुरुष,नेपाल कम्युनिष्ट पार्टी (एकीकृत मार्क्सवादी ले...,2598,सुर्य,ताप्लेजुङ,ताप्लेजुङ,कोशी प्रदेश,...,1,None,38,भुपेन्द्र थेबे,-,स्नातक,TU,0,0,ताप्लेजुङ सिरीजङ्घा गाउँपालिका मादिबुङ
1,341131,गजेन्द्र प्रसाद तुम्याङ लिम्बु,46,पुरुष,नेपाली काँग्रेस,2583,रुख,ताप्लेजुङ,ताप्लेजुङ,कोशी प्रदेश,...,1,None,46,पुर्ण बहादुर लिम्बु,राधिका लिङदेन तुम्याड,स्नातक,पाथीभरा वहुमुखी क्याम्पस,0,0,ताप्लेजुङ सिरीजङ्घा गाउँपालिका
2,339610,खेल प्रसाद बुडाक्षेत्री,50,पुरुष,नेपाली कम्युनिष्ट पार्टी,2557,पाँचकुने तारा,ताप्लेजुङ,ताप्लेजुङ,कोशी प्रदेश,...,1,None,50,शम्सेर बहादुर बुडाक्षेत्री,कुमारी बुडाक्षेत्री,I.s.c,TU,0,0,ताप्लेजुङ सिरीजङ्घा गाउँपालिका चिचिहाङ्ग
3,340459,बिरेन्द्र श्रेष्ठ,55,पुरुष,राष्ट्रिय स्वतन्त्र पार्टी,2528,घण्टी,ताप्लेजुङ,ताप्लेजुङ,कोशी प्रदेश,...,1,None,55,टेकबहादुर श्रेष्ठ,मानोलाक वुफाश्रि श्रेष्ठ,ईन्जिनेरिङ ग्राजुएड,युनिभर सिटी आर्मानिय,0,0,ताप्लेजुङ सिरीजङ्घा गाउँपालिका
4,340160,ईन्द्र प्रसाद थापा,63,पुरुष,राष्ट्रिय प्रजातन्त्र पार्टी,2604,हलो,कैलाली,ताप्लेजुङ,कोशी प्रदेश,...,1,None,63,नरजुङ्ग थापा,"इश्वरा, पु्ण्यवती देवी, प्रसाई, अधिकारी (थापा)",आई.ए.,पाथीभरा वहुमुखी क्याम्पस,0,0,ताप्लेजुङ मिक्वाखोला गाउँपालिका थापा गाउँ


In [15]:
stg_pratakhya_election_result = conn.execute("SELECT * FROM stg_pratakhya_election_result").fetchdf()
stg_pratakhya_election_result.head()

,CandidateName,Gender,Age,PartyID,SymbolID,SymbolName,CandidateID,StateName,PoliticalPartyName,ElectionPost,...,Samudaya,DOB,CTZDIST,FATHER_NAME,SPOUCE_NAME,QUALIFICATION,EXPERIENCE,OTHERDETAILS,NAMEOFINST,ADDRESS
0,योगेश कुमार भट्टराई,पुरुष,56,0,2,सूर्य,335800,कोशी प्रदेश,नेपाल कम्युनिष्ट पार्टी (एमाले),None,...,None,2023.05.21,ताप्लेजुंग,भिम लाल भट्टराई,विन्दा घिमिरे [भट्टराई],Masters,0,0,त्रिभुवन विश्वविद्यालय किर्तिपूर काठमाण्डौं,ताप्लेजुंग मैवाखोला गाउँपालिका सगरमाथा मार्ग
1,खेल प्रसाद बुडाक्षेत्री,पुरुष,46,0,3,गोलाकारभित्र हँसिया हथौडा,336693,कोशी प्रदेश,नेपाल कम्युनिष्ट पार्टी (माओवादी केन्द्र)(एकल ...,None,...,None,2032.08.12,ताप्लेजुंग,शम्सेर बहादुर बुडाक्षेत्री,कुमारी बुडाक्षेत्री,Proficiency Certificate,"शिक्षण, कृषी, राजनीती,",स्वच्छ प्रतिस्पर्धाबाट निर्वाचित जनप्रतिनिधीहर...,हात्तिसार क्याम्पस धरान,ताप्लेजुंग सिरीजङ्घा गाउँपालिका चिचिहाङ्ग
2,होम प्रसाद तुम्बाहाम्फे,पुरुष,56,0,56,दाप सहितको खुकुरी,334167,कोशी प्रदेश,संघीय लोकतान्त्रिक राष्ट्रिय मञ्च,None,...,None,2022.10.15,ताप्लेजुंग,दल बहादुर तुम्बाहाम्फे,विना तुम्बाहाम्फे,S.L.C.,0,0,भानुजन मा.वि. फुङलिङ-४,ताप्लेजुंग फुङलिङ नगरपालिका बोझे
3,चन्द्र मादेन,पुरुष,49,0,2278,ढल्केको छाता,336924,कोशी प्रदेश,"जनता समाजवादी पार्टी, नेपाल",None,...,None,2030.01.19,ताप्लेजुंग,विष्णु मादेन,प्रिया नेम्वाङ,Bachelor,,,मिनेसोटा युनिभरसिटी अमेरिका,ताप्लेजुंग फुङलिङ नगरपालिका हुलाक टोल
4,राम बहादुर कार्की,पुरुष,65,0,2279,हलो,336050,कोशी प्रदेश,राष्ट्रिय प्रजातन्त्र पार्टी,None,...,None,2014.05.15,ताप्लेजुंग,सन्त बहादुर कार्की,विमला नेयोहाङ्ग,Bachelor,राजनिती/ समाजसेवा,,"त्रिभुवान विश्वविद्यालय, काठमाण्डौ",ताप्लेजुंग फुङलिङ नगरपालिका


In [14]:
stg_current_candidates.columns

Index(['CandidateID', 'CandidateName', 'AGE_YR', 'Gender',
       'PoliticalPartyName', 'SYMBOLCODE', 'SymbolName', 'CTZDIST',
       'DistrictName', 'StateName', 'STATE_ID', 'SCConstID', 'ConstName',
       'TotalVoteReceived', 'R', 'E_STATUS', 'DOB', 'FATHER_NAME',
       'SPOUCE_NAME', 'QUALIFICATION', 'NAMEOFINST', 'EXPERIENCE',
       'OTHERDETAILS', 'ADDRESS'],
      dtype='str')

In [ ]:
# Get all uniques based on 'PoliticalPartyName','SymbolName' 
unique_parties_current = stg_current_candidates[['PoliticalPartyName', 'SymbolName']].drop_duplicates().reset_index(drop=True)
# exclude where party name is 'स्वतन्त्र'
unique_parties_current = unique_parties_current[unique_parties_current['PoliticalPartyName'] != 'स्वतन्त्र']
unique_parties_current

In [ ]:
stg_pratakhya_election_result.columns

In [ ]:
unique_parties_result = stg_pratakhya_election_result[['PoliticalPartyName', 'SymbolName']].drop_duplicates().reset_index(drop=True)
# exclude where party name is 'स्वतन्त्र'
unique_parties_result = unique_parties_result[unique_parties_result['PoliticalPartyName'] != 'स्वतन्त्र']
unique_parties_result

In [ ]:
# Join both dataframes to see differences do a full outer join on PoliticalPartyName and SymbolName
merged_parties = unique_parties_current.merge(unique_parties_result, on=['SymbolName'], how='outer', suffixes=('_current', '_previous'))
merged_parties

In [ ]:
# export merged_parties to a csv file apply proper encoding to handle nepali characters
merged_parties.to_csv('party_name_symbol_comparison.csv', index=False, encoding='utf-8-sig')

In [ ]:
def find_vagaute_from_party(past_election_result, current_candidates):
    candidates = past_election_result
    merged = candidates.merge(current_candidates, on=['CandidateName', 'FATHER_NAME'], how='left', suffixes=('_past', '_current'))
    return merged[['CandidateName', 'PoliticalPartyName_past', 'SymbolName_past', 'PoliticalPartyName_current', 'SymbolName_current']]

vagaute_df = find_vagaute_from_party(stg_pratakhya_election_result, stg_current_candidates)
vagaute_df.to_csv('vagaute_party_comparison.csv', index=False, encoding='utf-8-sig')
vagaute_df

In [ ]:
# Find potential Party name change or mergers based on matching SymbolName but different PoliticalPartyNames

vagaute_df['Potential_Merger'] = np.where((vagaute_df['PoliticalPartyName_past'] != vagaute_df['PoliticalPartyName_current']) & 
                                         (vagaute_df['PoliticalPartyName_current'].notnull()), True, False)

# If more than 50% of candidates from a past part are in new same party then makr that as potential merger
party_merger_summary = vagaute_df.groupby(['PoliticalPartyName_past', 'PoliticalPartyName_current']).agg(
    total_candidates_past=('CandidateName', 'count'),
    merged_candidates=('Potential_Merger', 'sum')
).reset_index()
party_merger_summary['Merger_Flag'] = np.where(
    party_merger_summary['merged_candidates'] / party_merger_summary['total_candidates_past'] > 0.5, True, False
)
party_merger_summary.to_csv('vagaute_party_merger_summary.csv', index=False, encoding='utf-8-sig')

In [ ]:
stg_current_candidates.columns

In [ ]:
stg_current_candidates[['QUALIFICATION']].drop_duplicates().reset_index(drop=True)
# save it as csv
stg_current_candidates[['QUALIFICATION']].drop_duplicates().reset_index(drop=True).to_csv('current_candidates_qualifications.csv', index=False, encoding='utf-8-sig')

In [ ]:
stg_current_candidates[['QUALIFICATION']].drop_duplicates().reset_index(drop=True).to_json('current_candidates_qualifications.json', orient='records', force_ascii=False)

In [ ]:
conn.close()

In [12]:
qualification_levels = pd.read_json('qualification_level_mapping.json', orient='records')
qualification_levels.to_csv('qualification_level_mapping.csv', index=False, encoding='utf-8-sig')

In [16]:
# check the qualifications from QUALIFICATION stg_pratakhya_election_result and see if we are missing any qualifications in our mapping. Read the csv back, compare and give me the missing ones in json format.
past_qualifacionts = stg_pratakhya_election_result[['QUALIFICATION']].drop_duplicates().reset_index(drop=True)
# read the mapping csv
qualification_mapping = pd.read_csv('election/seeds/qualification_level_mapping.csv', encoding='utf-8-sig')

# show the qualifications from past elections that are not in the mapping
missing_qualifications = past_qualifacionts[~past_qualifacionts['QUALIFICATION'].isin(qualification_mapping['QUALIFICATION'])]
missing_qualifications.to_json('missing_qualifications.json', orient='records', force_ascii=False)
missing_qualifications

,QUALIFICATION
1,Proficiency Certificate
5,"आई, एल."
7,प्रमाणपत्र तह
12,UNDER SLC
14,+2
...,...
300,+१२
301,+२
302,"BK, MA"
304,"BA, B.ed"


In [17]:
missing_qualifications_mapping= pd.read_json('missing_qualification_mapping.json', orient='records')
missing_qualifications_mapping.to_csv('missing_qualification_mapping.csv', index=False, encoding='utf-8-sig')

In [18]:
con.close()

NameError: name 'con' is not defined